# Spaceship Titanic: EDA & Feature Engineering 발표자료

이 노트북은 Spaceship Titanic 경진대회의 데이터 분석(EDA) 및 특성 공학(Feature Engineering) 과정을 설명하기 위한 발표 자료입니다.

## 목차
1. **데이터 로드 및 개요**
2. **탐색적 데이터 분석 (EDA)**
3. **결측치 처리 (Missing Values Imputation)**
4. **파생 변수 생성 (Feature Engineering)**
5. **인코딩 및 스케일링 (Encoding & Scaling)**
6. **데이터 분리 (Data Split)**

## 1. 데이터 로드 및 개요

먼저 필요한 라이브러리를 불러오고, 학습 데이터(`train.csv`)와 테스트 데이터(`test.csv`)를 로드합니다.
두 데이터를 합쳐서(`all_data`) 전처리를 일괄적으로 수행하겠습니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

# 전처리를 위해 데이터 합치기
all_data = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)

# 데이터 확인
print(f"전체 데이터 크기: {all_data.shape}")
all_data.head()

## 2. 탐색적 데이터 분석 (EDA)

데이터의 특징을 파악하기 위해 주요 변수들에 대한 시각화와 분석을 진행합니다.

### 지출 금액 (TotalSpending) 분석
승객들이 선내에서 사용한 금액의 총합(`TotalSpending`)을 계산하고, 지출 여부에 따른 생존율을 확인해봅니다.

In [ ]:
# 지출 항목 합계 계산
spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
all_data['TotalSpending'] = all_data[spending_cols].sum(axis=1)

# 지출 그룹 생성 (지출 없음, 소액, 고액)
def making_spending_group(x):
    if x == 0:
        return '0원 구간'
    elif x <= 1200:
        return '중간 지출 구간'
    else:
        return '고액 지출 구간'

all_data['SpendingGroup'] = all_data['TotalSpending'].apply(making_spending_group)

# 지출 그룹별 생존율 시각화 (Train 데이터만 사용)
plt.figure(figsize=(10, 6))
sns.countplot(data=all_data[:len(train_df)], x='SpendingGroup', hue='Transported')
plt.title('Transported count by Spending Group')
plt.show()

### 고향 행성 (HomePlanet) 분석
출신 행성별 승객 수와 생존율을 비교합니다.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=all_data[:len(train_df)], x='HomePlanet', hue='Transported')
plt.title('Transported count by HomePlanet')
plt.show()

### 동면 여부 (CryoSleep) 분석
동면 상태였던 승객들의 생존율이 확연히 높은지 확인합니다. 이는 매우 중요한 예측 변수가 될 것입니다.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=all_data[:len(train_df)], x='CryoSleep', hue='Transported')
plt.title('Transported count by CryoSleep')
plt.show()

## 3. 결측치 처리 (Missing Values Imputation)

데이터에 존재하는 결측치를 다양한 논리와 통계적 방법을 사용하여 채워보겠습니다.

### 3-1. CryoSleep (동면 여부) 결측치 처리
- 지출 내역이 있다면 깨어있었으므로 `False`
- 지출이 전혀 없다면 동면 중이었을 가능성이 높으므로 `True`

In [ ]:
all_data.loc[(all_data['CryoSleep'].isna()) & (all_data[spending_cols].sum(axis=1) > 0), 'CryoSleep'] = False
all_data.loc[(all_data['CryoSleep'].isna()) & (all_data[spending_cols].sum(axis=1) == 0), 'CryoSleep'] = True

print(f"CryoSleep 남은 결측치: {all_data['CryoSleep'].isna().sum()}개")

### 3-2. Age (나이) 결측치 처리
나이는 전체 승객의 **중앙값(Median)**으로 채우고, 이를 바탕으로 `AgeGroup` 파생 변수를 생성합니다.

In [ ]:
age_median = all_data['Age'].median()
all_data['Age'] = all_data['Age'].fillna(age_median)

def update_age_group(age):
    if age <= 4: return 'Baby'
    elif age <= 12: return 'Child'
    elif age <= 19: return 'Teenager'
    elif age <= 40: return 'Adult'
    elif age <= 60: return 'Middle Aged'
    else: return 'Senior'

all_data['AgeGroup'] = all_data['Age'].apply(update_age_group)
print("Age 결측치 처리 완료 및 AgeGroup 생성")

### 3-3. VIP 결측치 처리
VIP 여부는 다음 조건들을 고려하여 `False`일 확률이 높은 경우를 먼저 채웁니다.
- 지출액이 0원인 경우
- 19세 이하 미성년자
- 고향이 Earth인 경우
- 남은 결측치는 최빈값인 `False`로 채웁니다.

In [ ]:
all_data.loc[(all_data['VIP'].isna()) & (all_data['TotalSpending'] == 0), 'VIP'] = False
all_data.loc[(all_data['VIP'].isna()) & (all_data['Age'] <= 19), 'VIP'] = False
all_data.loc[(all_data['VIP'].isna()) & (all_data['HomePlanet'] == 'Earth'), 'VIP'] = False
all_data['VIP'] = all_data['VIP'].fillna(False).astype(bool)

print(f"VIP 남은 결측치: {all_data['VIP'].isna().sum()}개")

### 3-4. Destination (목적지) 결측치 처리
가장 많은 승객이 향하는 최빈값(`TRAPPIST-1e`)으로 채웁니다.

In [ ]:
dest_mode = all_data['Destination'].mode()[0]
all_data['Destination'] = all_data['Destination'].fillna(dest_mode)
print(f"Destination 남은 결측치: {all_data['Destination'].isna().sum()}개")

### 3-5. Cabin, Surname, HomePlanet (그룹 정보 활용)
- `PassengerId`에서 `Group` 정보를 추출합니다.
- 같은 그룹원은 가족이나 동행일 확률이 높으므로 `Surname`, `HomePlanet`, `Cabin` 정보를 공유할 가능성이 큽니다.
- 이를 이용하여 결측치를 보간합니다.

In [ ]:
# PassengerId에서 Group 추출, Name에서 Surname 추출
all_data['Group'] = all_data['PassengerId'].apply(lambda x: x.split('_')[0])
all_data['Surname'] = all_data['Name'].apply(lambda x: x.split()[-1] if pd.notna(x) else np.nan)

# Surname 보정 (그룹 내 공유)
all_data['Surname'] = all_data.groupby('Group')['Surname'].ffill().bfill()

# HomePlanet 보정 (그룹 및 성씨 활용)
all_data['HomePlanet'] = all_data.groupby('Group')['HomePlanet'].ffill().bfill()
home_map = all_data.dropna(subset=['HomePlanet']).groupby('Surname')['HomePlanet'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
all_data['HomePlanet'] = all_data['HomePlanet'].fillna(all_data['Surname'].map(home_map))
all_data['HomePlanet'] = all_data['HomePlanet'].fillna(all_data['HomePlanet'].mode()[0])

# Cabin 파생 변수 생성 및 보정
all_data['Deck'] = all_data['Cabin'].apply(lambda x: x.split('/')[0] if pd.notna(x) else np.nan)
all_data['Num'] = all_data['Cabin'].apply(lambda x: x.split('/')[1] if pd.notna(x) else np.nan)
all_data['Side'] = all_data['Cabin'].apply(lambda x: x.split('/')[2] if pd.notna(x) else np.nan)

# 같은 그룹 내 Deck, Side, Num 공유
all_data['Deck'] = all_data.groupby('Group')['Deck'].ffill().bfill()
all_data['Side'] = all_data.groupby('Group')['Side'].ffill().bfill()
all_data['Num'] = pd.to_numeric(all_data['Num'], errors='coerce')
all_data['Num'] = all_data.groupby('Group')['Num'].ffill().bfill()

print("그룹 정보를 활용한 결측치 처리 완료")

## 4. 추가 전처리 (이상치 처리 및 컬럼 정리)

- 동면 상태(`CryoSleep=True`)인 승객의 지출액은 0원이어야 하므로 강제 보정합니다.
- 불필요한 컬럼(`PassengerId`, `Name`, `Cabin` 등)을 제거합니다.

In [ ]:
# 이상치 처리
all_data.loc[all_data['CryoSleep'] == 1, spending_cols] = 0

# 중복 제거
all_data = all_data.drop_duplicates()

# 불필요한 컬럼 삭제
drop_cols = ['PassengerId', 'Name', 'Cabin', 'Surname', 'Group']
all_data = all_data.drop(columns=drop_cols, errors='ignore')

print("최종 컬럼 목록:", all_data.columns.tolist())

## 5. 인코딩 및 스케일링

- 범주형 변수: One-Hot Encoding
- 수치형 변수: StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

# 원-핫 인코딩
encoding_cols = ['HomePlanet', 'Destination', 'AgeGroup', 'SpendingGroup', 'Deck', 'Side']
all_data = pd.get_dummies(all_data, columns=encoding_cols)

# 불리언 타입 정수형 변환
all_data = all_data.astype(int)

# 스케일링
scaling_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalSpending', 'Num']
# GroupSize, FamilySize가 있다면 추가
if 'GroupSize' in all_data.columns: scaling_cols.append('GroupSize')
if 'FamilySize' in all_data.columns: scaling_cols.append('FamilySize')

scaler = StandardScaler()
all_data[scaling_cols] = scaler.fit_transform(all_data[scaling_cols])

print("인코딩 및 스케일링 완료")
all_data.head()

## 6. 데이터 분리

전처리가 완료된 데이터를 다시 학습용(`X`, `y`)과 테스트용(`test_final`)으로 분리합니다.

In [ ]:
from sklearn.model_selection import train_test_split

# Train/Test 분리
X = all_data[:len(train_df)].drop(columns=['Transported'])
y = all_data[:len(train_df)]['Transported']
test_final = all_data[len(train_df):].drop(columns=['Transported'])

# 검증 검증용 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")